<div style="width:100%; background-color: #000041"><a target="_blank" href="bit.ly/45U3TvK"><img src="assets/YugabyteDB_DSS-Virtual_LinkedIn-Cover_1584x396.jpg" /></a></div>

<br>

> **Migrate from MySQL to Distributed, Highly Available PostgreSQL!**
>
> Registration is free  [Distributed SQL Summit](bit.ly/45U3TvK).
>

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Review the source database, *which you must do next*
- ☑️ Migration workflow (1)

### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### Import the notebook variables 

> ‼️ Important
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -r MY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# meta commands

./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dn"
./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dt sakila."
./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dv sakila."

---
# Review the MySQL database for migration to Highly Available PostgreSQL

Here is the related schema for the Sakila database.

<div style="width:100%; background-color: #000041"><a target="_blank" href="https://dev.mysql.com/doc/sakila/en/sakila-structure.html"><img src="assets/sakila-schema.png" /></a></div>

Using `mysql`, you can execute MySQL statements.  The following commands are MySQL shell commands. 

To review the **tables** in the MySQL database, run the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

mysql -h 0 -u root ${DB_NAME} -e "SHOW TABLE STATUS; "  

To review the **views** in the MySQL database, run the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

# terminate connections, drop, and create
mysql -h 0 -u root ${DB_NAME} -e "SHOW FULL TABLES WHERE Table_Type LIKE 'VIEW';"


To view all the **indexes** for the tables, run the following the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

# terminate connections, drop, and create
mysql -h 0 -u root ${DB_NAME} -e "SELECT DISTINCT TABLE_NAME, INDEX_NAME FROM INFORMATION_SCHEMA.STATISTICS WHERE TABLE_SCHEMA = '${DB_NAME}'; " 

To review the **user-defined functions** in the MySQL database, run the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

# terminate connections, drop, and create
mysql -h 0 -u root ${DB_NAME} -e "SHOW FUNCTION STATUS WHERE db = '${DB_NAME}'; "  

To review the **triggers** in the MySQL database, run the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

# terminate connections, drop, and create
mysql -h 0 -u root ${DB_NAME} -e "SHOW TRIGGERS;"

To review the **stored procedures** in the MySQL database, run the following cell:

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

# terminate connections, drop, and create
mysql -h 0 -u root ${DB_NAME} -e "SHOW PROCEDURE STATUS WHERE db = '${DB_NAME}'; "   


---
# Migration workflow: Prepare


<div style="width:100%; background-color: #000041"><img src="assets/Migration_Workflow_01.png" /></div>

## Install YB-Voyager

Good news! This notebook environment in Gitpod already has YB-Voyager installed.  If you are curious about how to install YugabyteDB Voyager in your environment, check out these resources:
- [Download YugabteDB Voyager](https://download.yugabyte.com/migrate)
- [Install YugabyteDB Voyager](https://docs.yugabyte.com/preview/yugabyte-voyager/install-yb-voyager/)

## Prepare MySQL source

In [ ]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_MYSQL_DB_NAME
db_port=MY_MYSQL_PORT

# sql statements for grants for ybvoyager user
SQL_STATEMENTS = {} 

SQL_STATEMENTS['create_user_1'] = ("CREATE USER 'ybvoyager'@'127.0.0.1' IDENTIFIED WITH  mysql_native_password BY 'Yugabyte#1'")
#SQL_STATEMENTS['create_user_2'] = ("CREATE USER 'ybvoyager'@'locahost' IDENTIFIED WITH  mysql_native_password BY 'Yugabyte#1'")

SQL_STATEMENTS['grant_process_1'] = ("GRANT PROCESS ON *.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_process_2'] = ("GRANT PROCESS ON *.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_select_1'] = ("GRANT SELECT ON sakila.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_select_2'] = ("GRANT SELECT ON sakila.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_show_view_1'] = ("GRANT SHOW VIEW ON sakila.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_show_view_2'] = ("GRANT SHOW VIEW ON sakila.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_trigger_1'] = ("GRANT TRIGGER ON sakila.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_trigger_2'] = ("GRANT TRIGGER ON sakila.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_flush_1'] = ("GRANT FLUSH_TABLES ON *.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_flush_2'] = ("GRANT FLUSH_TABLES ON *.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_replication_1'] = ("GRANT REPLICATION CLIENT ON *.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_replication_2'] = ("GRANT REPLICATION CLIENT ON *.* TO 'ybvoyager'@'localhost'")

SQL_STATEMENTS['grant_routine_1'] = ("GRANT SHOW_ROUTINE ON *.* TO 'ybvoyager'@'127.0.0.1'")
#SQL_STATEMENTS['grant_routine_2'] = ("GRANT SHOW_ROUTINE ON *.* TO 'ybvoyager'@'localhost'")


# Create connection
cnx = mysql.connector.connect(user='root',
                              host='127.0.0.1',
                              database=db_name)
cursor = cnx.cursor()

# Execute statements
for sql_name in SQL_STATEMENTS:
    sql_statement = SQL_STATEMENTS[sql_name]
    try:
        print("Running sql statement  {}: ".format(sql_name), end='')
        cursor.execute(sql_statement)
    except mysql.connector.Error as err:
        print(err.msg)
    else:
        print("OK")

cursor.close()

## Prepare YugabyteDB target

Create the target database in your YugabyteDB cluster. The database name can be the same or different from the source database name. If the target database name is not provided, yb-voyager assumes the target database name to be yugabyte. If you do not want to import in the default yugabyte database, specify the name of the target database name using the --target-db-name argument to the yb-voyager import commands.

```
CREATE DATABASE target_db_name;
``````

### Create a user with SUPERUSER role.

For a local YugabyteDB cluster or YugabyteDB Anywhere, create a user and role with the superuser privileges using the following command:

```
CREATE USER ybvoyager SUPERUSER PASSWORD 'password';
```

For YugabyteDB Managed, create a user with yb_superuser role using the following command:

```
CREATE USER ybvoyager PASSWORD 'password';
GRANT yb_superuser TO ybvoyager;
```

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# select version

./bin/ysqlsh -d yugabyte -c "drop database if exists ${DB_NAME};"  
./bin/ysqlsh -d yugabyte -c "create database ${DB_NAME};"  
./bin/ysqlsh -d yugabyte -c "drop user if exists ybvoyager;"  
./bin/ysqlsh -d yugabyte -c "create user ybvoyager SUPERUSER PASSWORD 'password';"  

./bin/ysqlsh -d yugabyte -c "\l"  

---
# Step 3: Export the database schema

In MySQL, there is no difference between a schema and a database. In fact, the `CREATE SCHEMA` command is a synonym for the `CREATE DATABASE` command. To learn more, see [MySQL CREATE DATABASE Statement](https://dev.mysql.com/doc/refman/8.0/en/create-database.html).

In PostgreSQL, a database contains one or more named schemas, which in turn contain tables and other kinds of named objects, including data types, functions, and so on. In fact, the same object name can be used in different schemas without conflict. For example, both `my_schema` and `your_schema` can contain the same table, `table_shared`. 

Unlike databases, schemas are not rigidly separated in PostgreSQL. A user can access objects in any of the schemas in the database they are connected to, as long as they user has the required privileges to do so.

In this step, you will convert the MySQL database to a PostgreSQL format using the `yb-voyager export schema` command. 

Begin by creating an export directory, `export-dir`.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"

NOTEBOOK_VOYAGER_TMP=${1}
echo $NOTEBOOK_VOYAGER_TMP
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir
mkdir $NOTEBOOK_VOYAGER_TMP/export-dir

Next, export the schema using the `yb-voyager` CLI.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01"  "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager export schema --start-clean -y --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
        --source-db-type "MYSQL" \
        --source-db-host ${DB_HOST} \
        --source-db-port  ${DB_PORT} \
        --source-db-name  ${DB_NAME} \
        --source-db-user "ybvoyager" \
        --source-db-password "Yugabyte#1" \ 

List folders and files:
- `reports` contains the generated Schema Analysis Report.
- `schema` contains the source database schema translated to PostgreSQL. The schema is partitioned into smaller files by the schema object type such as tables, views, and so on.
- `data` contains `.TSV` (Tab Separated Values) files that are passed to the `COPY` command on the target database.
- `metainfo` and `temp` are internal to by yb-voyager
- `log/yb-voyager.log` contains log messages

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

ls $NOTEBOOK_VOYAGER_TMP/export-dir

Review the Voyager log.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat $NOTEBOOK_VOYAGER_TMP/export-dir/logs/yb-voyager.log

gp open $NOTEBOOK_VOYAGER_TMP/export-dir/logs/yb-voyager.log

 > Important!
 >
 > 👆 In the above output cell, select **Adjust cell output settings...**. 
 >
 > In the **Settings** tab, in `Notebook > Output: Scrolling`, select the checkbox ✅.
 >
 > In `Notebook > Output: Text Line Limit`, enter `200`.
 >
 > To save, close the **Settings** tab.
 > 
 > Well done! 🙌 

---

## 😊 Next up!
Continue your learning by opening the next notebook, `03_Migration_Workflow_Schema_1.ipynb`. 

Or, if using GitPod, run the following cell:

In [ ]:
%%bash
gp open '03_Migration_Workflow_Schema_1.ipynb'